In [1]:
import pandas as pd
import requests
import re
from api_key import census_api_key

# Load client data
file_path = "./resources/Cumulative Count of Clients-Final.csv"
df = pd.read_csv(file_path)

# Ensure column names are stripped of whitespace
df.rename(columns=lambda x: x.strip(), inplace=True)

C:\Users\fedo3\AppData\Local\Temp\ipykernel_13940\2665262675.py:8: DtypeWarning: Columns (17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [2]:
print(df.columns)

Index(['Client Number/ID', 'Participation Record ID', 'Date of Birth',
       'Age at Start of Reporting Period', 'Age Today',
       'Is this a caregiver or client profile?', 'Gender', 'Ethnicity',
       'Date of Participation', 'Program Location', 'Annual Family Income_x',
       'Military Service', 'Date of Exit', 'Education',
       'College or Trade School Name/Major/Grad Date:', 'Zip', 'Program',
       'Precinct', 'City Council District', 'Program Location.1',
       'Type of Participation', 'Current School District', 'Therapist',
       'Annual Family Income_y', 'Min_Income', 'Max_Income',
       'Total Population', 'Poverty Rate', 'White', 'Black', 'Hispanic',
       'Extra_Column'],
      dtype='object')


In [6]:
import pandas as pd
import re

# Keep df as the full dataset, work on a subset for Annual Family Income
income_data = df[['Annual Family Income_x']].copy()  # Work on a copy, not overwrite df

# Function to clean and split income data
def clean_income(value):
    value = str(value).replace("$", "").replace(",", "").strip()  # Ensure it's a string first
    range_match = re.match(r"(\d+)-(\d+)", value)
    if range_match:
        return int(range_match.group(1)), int(range_match.group(2))
    
    more_match = re.match(r"(\d+) or more", value)
    if more_match:
        return int(more_match.group(1)), 100000  # Keep numeric

    return None, None  # Handle unexpected values

# Apply function to create min and max income columns
income_data[["Min_Income", "Max_Income"]] = income_data["Annual Family Income_x"].apply(lambda x: pd.Series(clean_income(x)))

# Convert Max_Income to numeric where possible
income_data["Max_Income"] = pd.to_numeric(income_data["Max_Income"], errors="coerce")

# Merge income data back into the full dataset
df = df.merge(income_data, left_index=True, right_index=True, how="left")
print(df.head())

   Client Number/ID  Participation Record ID Date of Birth  \
0               169                   426652    12/30/1974   
1               169                   426652    12/30/1974   
2               169                   426652    12/30/1974   
3               195                   317877    04/16/2007   
4               195                   317877    04/16/2007   

   Age at Start of Reporting Period  Age Today  \
0                                47         50   
1                                47         50   
2                                47         50   
3                                14         17   
4                                14         17   

  Is this a caregiver or client profile?  Gender        Ethnicity  \
0                            Caregiver 1  Female  Hispanic/Latino   
1                            Caregiver 1  Female  Hispanic/Latino   
2                            Caregiver 1  Female  Hispanic/Latino   
3                                 Client    Male  

In [7]:
# Ensure ZIP column exists
if 'Zip' not in df.columns:
    raise ValueError("Error: 'Zip' column not found. Check column names.")

# Convert ZIP column to numeric and clean it
df['Zip'] = pd.to_numeric(df['Zip'], errors='coerce')
df = df.dropna(subset=['Zip'])
df['Zip'] = df['Zip'].astype(int).astype(str)

print("ZIP codes cleaned successfully!")

ZIP codes cleaned successfully!


In [8]:
# Census API details
BASE_URL = "https://api.census.gov/data/2022/acs/acs5"
CENSUS_API_KEY = census_api_key  # Use the imported API key

In [9]:
# Variables to fetch (example: population, poverty rate, ethnicity breakdown)
census_vars = {
    "Total Population": "B01003_001E",  # Total population
    "Poverty Rate": "B17001_002E",  # Poverty rate
    "White": "B02001_002E",  # White
    "Black": "B02001_003E",  # Black or African American
    "Hispanic": "B03003_003E",  # Hispanic or Latino
    "American Indian or Alaska Native": "B02001_004E",  # American Indian or Alaska Native
    "Asian": "B02001_005E",  # Asian
    "Native Hawaiian or Pacific Islander": "B02001_006E",  # Native Hawaiian or Other Pacific Islander
    "Two or More Races": "B02001_007E",  # Two or More Races
    "Children (Under 18)": "B09001_001E"  # Total Population under 18 (children)
}

# Create a DataFrame to store Census data
census_data = []

# Fetch data for each ZIP code
for zip_code in df['Zip'].unique():
    params = {
        "get": ",".join(census_vars.values()),
        "for": f"zip code tabulation area:{zip_code}",
        "key": CENSUS_API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        values = data[1]  # First row contains headers, second row has values
        census_data.append([zip_code] + values)
    else:
        print(f"Failed to fetch data for ZIP {zip_code}")

Failed to fetch data for ZIP 78469
Failed to fetch data for ZIP 78711


In [10]:
# Check the number of columns in census_data
print(f"Number of columns in census_data: {len(census_data[0])}")

Number of columns in census_data: 12


In [11]:
# Ensure census_vars has the correct number of keys (6 keys)
census_columns = ["Zip"] + list(census_vars.keys()) + ["Extra_Column"]  # This should now contain 7 columns in total
print(f"census_columns length: {len(census_columns)}")  # Make sure it has 7 columns

# Create DataFrame for Census data
census_df = pd.DataFrame(census_data, columns=census_columns)

# Merge with client data (df should have a 'Zip' column)
df_merged = df.merge(census_df, on='Zip', how='left')

# Save the merged data for Tableau import
df_merged.to_csv("./resources/Cumulative Count of Clients-Final.csv", index=False)

# Print confirmation message
print("Merged data saved. Ready for Tableau import!")

census_columns length: 12
Merged data saved. Ready for Tableau import!
